In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import networkx as nx
import numpy as np
from lib.graph_local_classes import GraphStructure, GraphParams, InnerGraphSimulation

In [3]:
# create a fully connected graph structure
nodes = ["A", "B", "C", "D"]
edges = []
for node1 in nodes:
    for node2 in nodes:
        if node1 == node2:
            continue
        edges.append((node1, node2))
        
gs = GraphStructure(nodes, edges)

In [18]:
# graph parameters
gp = GraphParams(len(edges))
gp.sample()

{'lambda0': [0.062695847589069278],
 'mu': array([ 0.31801852,  0.15961772,  0.54798542,  7.77746455,  3.08578133,
         3.22103282,  0.14668106,  6.40482104,  0.15101429,  0.40205551,
         4.87640812,  0.04868184]),
 'n': 12,
 'names': (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11),
 'p': 0.8,
 'psi': array([ 0.0247242 ,  0.01993138,  0.0383348 ,  0.20915333,  0.06149181,
         0.14537558,  0.03194521,  0.10411709,  0.04865893,  0.01264925,
         0.1237423 ,  0.01151007]),
 'psi_shape': 1.0,
 'r': array([ 0.07774452,  0.12486947,  0.06995588,  0.02689222,  0.01992747,
         0.04513322,  0.21778689,  0.01625605,  0.32221404,  0.03146144,
         0.02537571,  0.23643449]),
 'r_shape': 1.0,
 'scale_free_bounds': (0.01, 100)}

In [14]:
gp.init_to_dict()

{'n': 12,
 'names': (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11),
 'p': 0.8,
 'psi_shape': 1.0,
 'r_shape': 1.0,
 'scale_free_bounds': (0.01, 100)}

In [24]:
# inner graph simulation
sim = InnerGraphSimulation(gs, gp)
# np.fromiter(sim.sample_iter_solely_first_events(1),dtype=float)
list(sim.sample_iter_solely_first_events(100))

[array([  0.,  inf,  inf,  inf]),
 array([ 0.        ,  0.55746228,  0.55558036,  0.58127898]),
 array([  0.,  inf,  inf,  inf]),
 array([ 0.        ,  0.07814821,  0.07988542,  0.11453661]),
 array([ 0.        ,  0.01720749,  0.02118645,  0.01627335]),
 array([ 0.        ,  0.08820208,         inf,  0.0864364 ]),
 array([  0.,  inf,  inf,  inf]),
 array([ 0.        ,         inf,         inf,  0.08913017]),
 array([ 0.        ,  0.04573008,  0.09786022,  0.02721278]),
 array([ 0.        ,  0.06524633,  0.06647246,  0.06506019]),
 array([  0.,  inf,  inf,  inf]),
 array([ 0.        ,  0.04268043,  0.05378362,  0.03570754]),
 array([ 0.        ,  0.04726835,  0.06324041,  0.04348623]),
 array([ 0.        ,  0.07921501,  0.01149287,  0.07915569]),
 array([  0.,  inf,  inf,  inf]),
 array([ 0.        ,  0.01937406,  0.02844581,  0.03530338]),
 array([  0.,  inf,  inf,  inf]),
 array([  0.,  inf,  inf,  inf]),
 array([ 0.        ,         inf,  0.07983968,         inf]),
 array([  0.,  inf

In [23]:
sim._all_events

[(0.0, 'A'), (0.081012416281642408, 'D'), (0.10307053000102422, 'A')]

In [ ]:
class GraphSimulation(object):
    
    def __init__(self, events):
        self.inner = 
        
    def likelihood(self, data):
        # compute likelihood given sample from 